In [259]:
import sys
sys.path.insert(0,'/home/charles/@@@@@_jupyter/script')
# stdout= sys.stdout
# reload(sys)
# sys.setdefaultencoding('utf-8')
# sys.stdout = stdout
from MyEnv import Get_MyEnv
from NLP_QuestionType import *
from NLP_JiebaSegmentor import JiebaSegmentor, Get_JiebaSegmentor

In [260]:
import re
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import uuid
from collections import namedtuple
import json
from pypinyin import pinyin, Style
import logging
from pyspark.sql.functions import udf, lit, lower, length

In [261]:
# encoding=UTF-8
# !flask/bin/python

from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import BatchStatement
import pandas as pd
from pyspark.sql.types import StringType, StructType, StructField


class CassandraType(object):
    PRODUCTION = 0
    TEST = 1
    TEST_DOCKER = 2


class CassandraDAO(object):

    # you have to install following items :
    # a. python-Cassandra driver
    # b. pyspark cassandra connector

    def __init__(self, type):
        #         print('runing father.__init__')
        if type == CassandraType.PRODUCTION:
            self.contact_points = ['192.168.95.127', '192.168.95.122']
            self.contact_points_str = "192.168.95.127,192.168.95.122"
        elif type == CassandraType.TEST:
            self.contact_points = ['192.168.0.90', '192.168.0.91', '192.168.0.92']
            self.contact_points_str = "192.168.0.90,192.168.0.91,192.168.0.92"
        else:
            self.contact_points = ['192.168.0.121', '192.168.0.122', '192.168.0.52']
            self.contact_points_str = "192.168.0.121,192.168.0.122,192.168.0.52"

        self.formatString = "org.apache.spark.sql.cassandra"
        self.username = "username"
        self.password = "password"
        self.cluster = None
        self.session = None
        self.createSession()

    def __del__(self):
        self.cluster.shutdown()

    def pandas_factory(self, colnames, rows):
        return pd.DataFrame(rows, columns=colnames)

    def createSession(self):
        print "contact_points = " + self.contact_points_str
        self.cluster = Cluster(
            contact_points=self.contact_points,  # random select a node
            #             load_balancing_policy = DCAwareRoundRobinPolicy(local_dc='datacenter1'),
            #         auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
        )
        self.session = self.cluster.connect()
        self.session.row_factory = self.pandas_factory
        self.session.default_fetch_size = 10000000
        # needed for large queries, otherwise driver will do pagination. Default is 50000.

    def getSession(self):
        return self.session

    def execCQL(self, keyspace, cql):
        """
        execute CQL
        """
        self.session.set_keyspace(keyspace)
        self.session.execute_async(cql)

    def execCQLSelect(self, keyspace, cql):
        """
        execute CQL, select only
        """

        self.session.set_keyspace(keyspace)

        #       cassandra ResultSet
        async_results = self.session.execute_async(cql)
        return async_results

    def execCQLCallBackAnysc(self, keyspace, cql, handle_success, handle_error):
        """
        execute CQL, if success => handle_success function, else handle_error
        """
        self.session.set_keyspace(keyspace)
        async_results = self.session.execute_async(cql)
        async_results.add_callbacks(handle_success, handle_error)

    def execCQLSelectToPandasDF(self, keyspace, cql):
        """
        execute CQL, select only, return Pandas DataFrame
        """

        self.session.set_keyspace(keyspace)

        #       cassandra ResultSet
        async_results = self.session.execute_async(cql)
        #         async_results = self.session.execute_async(cql)
        #       to Pandas DataFrame
        return async_results.result()._current_rows

    def execCQLSelectToDF(self, sqlContext, keyspace, cql):
        """
        execute CQL, select only, return Spark DataFrame
        """

        #       pandas dataframe to spark dataframe
        pandas_dataframe = self.execCQLSelectToPandasDF(keyspace, cql)
        if pandas_dataframe.empty:
            schema = StructType([])
            return sqlContext.createDataFrame([], schema)
        else:
            return sqlContext.createDataFrame(pandas_dataframe)

    def execCQLSelectToRDD(self, sqlContext, keyspace, cql):
        """
        execute CQL, select only, return Spark RDD
        """

        return self.execCQLSelectToDF(sqlContext, keyspace, cql).rdd.map(tuple)  # dataFrame to RDD

    @property
    def contactPoints(self):
        return self.contact_points

    @contactPoints.setter
    def contactPoints(self, contact_points):
        self.contact_points = contact_points

    @contactPoints.deleter
    def contactPoints(self):
        del self.contact_points

    # pyspark cassandra connector
    def readFromCassandraDF(self, sqlContext, keyspace, table):
        """
        read data from Cassandra, return Dataframe
        """

        return sqlContext.read \
            .format(self.formatString) \
            .options(table=table, keyspace=keyspace) \
            .option("spark.cassandra.connection.host", self.contact_points_str) \
            .load()

    def readFromCassandraRDD(self, sqlContext, keyspace, table):
        """
        read data from Cassandra, return RDD
        """

        df = sqlContext.read \
            .format(self.formatString) \
            .options(table=table, keyspace=keyspace) \
            .option("spark.cassandra.connection.host", self.contact_points_str) \
            .load()
        return df.rdd.map(tuple)  # dataFrame to RDD

    def saveToCassandraDF(self, dataFrame, keyspace, table, mode="error"):
        """
        Save data to Cassandra using DataFrame, select one mode to save
        
        SaveMode.ErrorIfExists (default) | "error"      When saving a DataFrame to a data source,
                                                        if data already exists, an exception is expected to be thrown.
        SaveMode.Append                  | "append"     When saving a DataFrame to a data source,
                                                        if data/table already exists, contents of the DataFrame are
                                                        expected to be appended to existing data.
        SaveMode.Overwrite               | "overwrite"  Overwrite mode means that when saving a DataFrame to a data source,
                                                        if data/table already exists, existing data is expected to be
                                                        overwritten by the contents of the DataFrame.
        SaveMode.Ignore                  | "ignore"     Ignore mode means that when saving a DataFrame to a data source,
                                                        if data already exists, the save operation is expected to not
                                                        save the contents of the DataFrame and to not change the
                                                        existing data. This is similar to a CREATE TABLE IF NOT EXISTS

                                                        in SQL.
        """

        dataFrame.write \
            .format(self.formatString) \
            .mode(mode) \
            .options(table=table, keyspace=keyspace) \
            .option("spark.cassandra.connection.host", self.contact_points_str) \
            .save()


In [262]:
c_dao = CassandraDAO(CassandraType.PRODUCTION)
t_dao = CassandraDAO(CassandraType.TEST)

In [263]:
jieba_path = "/nfs/aq_test/jieba/"
jieba_dict_path1 = jieba_path + "dict_taiwan.txt"
jieba_dict_path2 = jieba_path +"userdict.txt"
jieba_dict_path3 = jieba_path +"dict.txt.big"
jieba_dict_path4 = jieba_path +"dict.txt.small"
jieba_stopwords_path = jieba_path +"stopwords.txt"
segmentor = JiebaSegmentor(jieba_dict_path1, 
                           [jieba_dict_path2,jieba_dict_path3,jieba_dict_path4], 
                           stopwords = False, 
                           stopwords_path=jieba_stopwords_path)

In [264]:
DB_KEYSPACE = "nlp_keyspace"
dcard_article = "dcard_article"
dcard_response = "dcard_response"
dcard_article_relationship = "dcard_article_relationship"
dcard_response_relationship = "dcard_response_relationship"
dcard_article_food = "dcard_article_food"
dcard_response_food = "dcard_response_food"
dcard_article_response_food = "dcard_article_response_food"

In [265]:
data_df = t_dao.readFromCassandraDF(sqlContext, DB_KEYSPACE, dcard_article)
data_df.show(5)

In [266]:
data_df = data_df.select(data_df['article_id'], 
                         data_df['board'], 
                         data_df['crawl_date'], 
                         data_df['create_date'],
                         data_df['title'])
data_df = data_df.withColumn("title_clean", data_df['title'])
data_df = data_df.withColumnRenamed('create_date','article_create_date')
data_df = data_df.withColumnRenamed('crawl_date','article_crawl_date')
data_df
# a_df = a_df.withColumnRenamed('create_date','question_create_date')
# r_df = r_df.withColumn("response", r_df['content'])

DataFrame[article_id: string, board: string, article_crawl_date: timestamp, article_create_date: string, title: string, title_clean: string]

In [267]:
# def modify_column():
#     return "relationship"
# modify_column_udf = udf(modify_column)
# data_df = data_df.withColumn("board",
#                              modify_column_udf())
# data_df.show(5)

In [268]:
# replace_string = [u'問',u'圖',u'更',u'轉',u'更新',u'文長',
#                   u'閒聊',u'黑特',u'分享',u'慎入',u'圖多',
#                   u'有雷',u'心得',u'開箱',u'轉載',u'情報', u'急', u'轉']
# replace_symbo_string =[x + u" " for x in replace_string] + \
#                       [x + u":" for x in replace_string] + \
#                       [x + u"　" for x in replace_string] + \
#                       [x + u"：" for x in replace_string] + \
#                       [x + u")" for x in replace_string] + \
#                       [u"("+ x + u")" for x in replace_string] + \
#                       [x + u"）" for x in replace_string] + \
#                       [u"（"+ x + u"）" for x in replace_string]
# for r in replace_symbo_string:
#     print r

In [269]:
import re
def remove_question_tag(question):
    question = re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]|\\［.*?］|\\（.*?）|\\【.*?】|\\#.*? |\\#.*?#", u"", 
                      question)
    question = re.sub(u'(Re:|re:)', u'', question)
    replace_string = [u'問',u'圖',u'更',u'轉',u'更新',u'文長',
                      u'閒聊',u'黑特',u'分享',u'慎入',u'圖多',
                      u'有雷',u'心得',u'開箱',u'轉載',u'情報', u'急', u'轉']
    replace_symbo_string =[x + u" " for x in replace_string] + \
                          [x + u":" for x in replace_string] + \
                          [x + u")" for x in replace_string] + \
                          [x + u"　" for x in replace_string] + \
                          [x + u"：" for x in replace_string] + \
                          [x + u"）" for x in replace_string]
    
    for s in replace_symbo_string:
        question = question.replace(s, u'')
    for s in replace_string:
        question = question.replace(s, u'')

    return question

remove_question_tag_udf = udf(remove_question_tag)
data_df = data_df.withColumn("title_clean", 
                             remove_question_tag_udf(data_df['title']))
data_df.show(5)

In [270]:
data_df = data_df.dropna()

In [271]:
# jieba = Get_JiebaSegmentor()

In [272]:
# qt = QuestionTypeCheck(segmentor)
# tp = qt.check_question_type("今天爲何會下雨")
# tp.target, tp.label

In [273]:
data_df.select("title", "title_clean").show(50)

In [274]:
# data_df = data_df.filter(data_df.board == "food")
# data_df.count()

In [275]:
data_pd = data_df.toPandas()

In [276]:
import emoji
# def remove_emoji(question):
    
#     question = re.sub(u':\S+?:', u'', emoji.demojize(question))
#     return question

# remove_emoji_udf = udf(remove_emoji)
# data_df = data_df.withColumn("title_clean", 
#                              remove_emoji_udf(data_df['title_clean']))

def remove_emoji(pd):
    return re.sub(u':\S+?:', u'', emoji.demojize(pd['title_clean']))

data_pd['title_clean'] = data_pd.apply(remove_emoji, axis=1)
data_pd

,article_id,board,article_crawl_date,article_create_date,title,title_clean
0,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇
1,232673887,food,2020-03-31 18:23:29.408,2019-12-12T12:38:28.706Z,暴飲暴食好罪惡可是好快樂,暴飲暴食好罪惡可是好快樂
2,233088675,food,2020-03-31 15:21:06.420,2020-02-16T15:02:19.734Z,吃貨日記。台中｜CAMPUS CAFE、宮原眼科、老賴紅茶,吃貨日記。台中｜CAMPUS CAFE、宮原眼科、老賴紅茶
3,232464517,food,2020-04-06 09:46:51.685,2019-11-08T15:08:23.420Z,歌林美食鍋的問題,歌林美食鍋的題
4,231549818,food,2020-04-07 10:01:08.695,2019-06-22T10:53:40.357Z,#台南 推薦火車站附近的果汁店,推薦火車站附近的果汁店
5,233336486,relationship,2020-03-30 15:48:24.131,2020-03-25T01:06:53.768Z,放下一個人好難,放下一個人好難
6,232761831,food,2020-03-31 18:07:43.666,2019-12-25T20:46:08.724Z,krispy kreme甜甜圈,krispy kreme甜甜圈
7,233304131,relationship,2020-03-30 16:45:22.653,2020-03-20T09:03:56.914Z,異地或異國戀有沒有未來,異地或異國戀有沒有未來
8,233178362,relationship,2020-04-08 10:07:22.021,2020-03-02T18:16:20.231Z,感謝最初的你但真的好遺憾,感謝最初的你但真的好遺憾
9,232844039,food,2020-03-31 17:54:53.805,2020-01-07T11:35:30.900Z,甘泉O麵的部分,甘泉O麵的部分


In [277]:
qt = QuestionTypeCheck(segmentor)
def question_type(pd):
    return ""
#     return qt.check_question_type(pd['title_clean']).label

data_pd['question_type'] = data_pd.apply(question_type, axis=1)
data_pd

,article_id,board,article_crawl_date,article_create_date,title,title_clean,question_type
0,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,
1,232673887,food,2020-03-31 18:23:29.408,2019-12-12T12:38:28.706Z,暴飲暴食好罪惡可是好快樂,暴飲暴食好罪惡可是好快樂,
2,233088675,food,2020-03-31 15:21:06.420,2020-02-16T15:02:19.734Z,吃貨日記。台中｜CAMPUS CAFE、宮原眼科、老賴紅茶,吃貨日記。台中｜CAMPUS CAFE、宮原眼科、老賴紅茶,
3,232464517,food,2020-04-06 09:46:51.685,2019-11-08T15:08:23.420Z,歌林美食鍋的問題,歌林美食鍋的題,
4,231549818,food,2020-04-07 10:01:08.695,2019-06-22T10:53:40.357Z,#台南 推薦火車站附近的果汁店,推薦火車站附近的果汁店,
5,233336486,relationship,2020-03-30 15:48:24.131,2020-03-25T01:06:53.768Z,放下一個人好難,放下一個人好難,
6,232761831,food,2020-03-31 18:07:43.666,2019-12-25T20:46:08.724Z,krispy kreme甜甜圈,krispy kreme甜甜圈,
7,233304131,relationship,2020-03-30 16:45:22.653,2020-03-20T09:03:56.914Z,異地或異國戀有沒有未來,異地或異國戀有沒有未來,
8,233178362,relationship,2020-04-08 10:07:22.021,2020-03-02T18:16:20.231Z,感謝最初的你但真的好遺憾,感謝最初的你但真的好遺憾,
9,232844039,food,2020-03-31 17:54:53.805,2020-01-07T11:35:30.900Z,甘泉O麵的部分,甘泉O麵的部分,


In [278]:
data_pd['article_crawl_date'] = pd.Series(data_pd['article_crawl_date'].dt.to_pydatetime(), dtype=object)
data_pd.info()

In [279]:
# title 並非完整文句
len(data_pd), len(data_pd[data_pd.question_type != "other"])
# data_df.filter(data_df.question_type == "other").count()

(17725, 17725)

In [280]:
data_pd[data_pd.question_type != "other"]

,article_id,board,article_crawl_date,article_create_date,title,title_clean,question_type
0,231747478,food,2020-04-06 16:50:45.114000,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,
1,232673887,food,2020-03-31 18:23:29.408000,2019-12-12T12:38:28.706Z,暴飲暴食好罪惡可是好快樂,暴飲暴食好罪惡可是好快樂,
2,233088675,food,2020-03-31 15:21:06.420000,2020-02-16T15:02:19.734Z,吃貨日記。台中｜CAMPUS CAFE、宮原眼科、老賴紅茶,吃貨日記。台中｜CAMPUS CAFE、宮原眼科、老賴紅茶,
3,232464517,food,2020-04-06 09:46:51.685000,2019-11-08T15:08:23.420Z,歌林美食鍋的問題,歌林美食鍋的題,
4,231549818,food,2020-04-07 10:01:08.695000,2019-06-22T10:53:40.357Z,#台南 推薦火車站附近的果汁店,推薦火車站附近的果汁店,
5,233336486,relationship,2020-03-30 15:48:24.131000,2020-03-25T01:06:53.768Z,放下一個人好難,放下一個人好難,
6,232761831,food,2020-03-31 18:07:43.666000,2019-12-25T20:46:08.724Z,krispy kreme甜甜圈,krispy kreme甜甜圈,
7,233304131,relationship,2020-03-30 16:45:22.653000,2020-03-20T09:03:56.914Z,異地或異國戀有沒有未來,異地或異國戀有沒有未來,
8,233178362,relationship,2020-04-08 10:07:22.021000,2020-03-02T18:16:20.231Z,感謝最初的你但真的好遺憾,感謝最初的你但真的好遺憾,
9,232844039,food,2020-03-31 17:54:53.805000,2020-01-07T11:35:30.900Z,甘泉O麵的部分,甘泉O麵的部分,


In [281]:
# from pyspark.sql.types import StringType,IntegerType,TimestampType,StructType,StructField
# fields =[StructField("article_id", StringType(), True) ,
#         StructField("board", StringType(), True) ,
#         StructField("crawl_date", TimestampType(), True) ,
#         StructField("create_date", StringType(), True) ,
#         StructField("title", StringType(), True) , 
#         StructField("title_clean", StringType(), True) , 
#         StructField("question_type", StringType(), True)]
# schema = StructType(fields)
data_df = sqlContext.createDataFrame(data_pd)
data_df.show(150)

In [282]:
# food_article_df = data_df.filter(data_df.board == "food")
# relationship_article_df = data_df.filter(data_df.board == "relationship")
# food_article_df.count(),relationship_article_df.count()

In [283]:
# food_article_df.show(5)
article_df = data_df
article_df.show(5)

In [284]:
# t_dao.saveToCassandraDF(food_data_df, DB_KEYSPACE, dcard_article_food, 'overwrite')
# t_dao.saveToCassandraDF(relationship_data_df, DB_KEYSPACE, dcard_article_relationship, 'overwrite')

In [285]:
# def question_type(response):
    
#     return  qt.check_question_type(input_text)

# qt = QuestionTypeCheck(jieba)

# question_type_udf = udf(question_type)
# data_df = data_df.withColumn("question_type", 
#                              question_type_udf(data_df['title_clean']))

In [286]:
# ppp

# response cleaning

In [287]:
data_df = t_dao.readFromCassandraDF(sqlContext, DB_KEYSPACE, dcard_response)
data_df.show(5)

In [288]:
# jennifer幫忙加入board
# data_df = data_df.withColumn("board", lit("relationship"))
# t_dao.saveToCassandraDF(data_df, DB_KEYSPACE, source_response, 'overwrite')

In [289]:
data_df = data_df.withColumnRenamed("content", 'response')
data_df = data_df.withColumnRenamed("article_uid", 'article_id_2')
data_df = data_df.withColumnRenamed('create_date','response_create_date')
data_df = data_df.withColumnRenamed('crawl_date','response_crawl_date')
data_df = data_df.withColumn("response_clean", data_df['response'])
# data_df = data_df.select('article_id', 
#                          'floor', 
#                          'board',
#                          'response', 
#                          'response_clean',
#                          'response_create_date',
#                          'response_crawl_date',
#                          'like_count')
data_df.show(5)

In [290]:
data_df.count()

174917

In [291]:
def response_too_long(response):
        
    if len(response) > 40:  
        return True 

    return False

# response_too_long_udf = udf(response_too_long)
# data_df = data_df.withColumn("response_too_long", 
#                              response_too_long_udf(data_df['response_clean']))
data_df = data_df.filter(length(data_df["response_clean"]) < 40)
data_df.show(5)

In [292]:
data_df.count()

132157

In [293]:
def has_url(response):
        
    rule = 'http://\S+|https://\S+'
    pattern = re.compile(rule)
    match = pattern.search(response)
    if match:  
       return True 

#     rule = '(.png|.jpg|.jpeg|.gif|.svg|.txt)'
#     pattern = re.compile(rule)
#     match = pattern.search(response)
#     if match:  
#        return True 
    
    return False

has_url_udf = udf(has_url)
data_df = data_df.withColumn("has_url", 
                             has_url_udf(data_df['response_clean']))
data_df.show(5)
# .limit(5).toPandas()

In [294]:
def has_reply_tag(response):
        
#     rule = '(B[\d+]|b[\d+]|[\d+]F|[\d+]f|[\d+]樓|樓上|樓下|樓主|原po|原PO|原Po|原波|[+1])'
    rule = '(b[\d+]|[\d+]f|[\d+]樓|樓上|樓下|樓主|原po|原波)'
    pattern = re.compile(rule)
    match = pattern.search(response)
    if match:  
        return True 

#     rule = 'http://\S+|https://\S+'
#     pattern = re.compile(rule)
#     match = pattern.search(response)
#     if match:  
#        return True 

    return False

has_reply_tag_udf = udf(has_reply_tag)
data_df = data_df.withColumn("has_reply_tag", 
                             has_reply_tag_udf(lower(data_df['response_clean'])))
data_df.show(5)

In [295]:
# test_df = data_df.filter(data_df.has_reply_tag == True)
# # test_df.show(5)
# test_df[test_df['article_id'], test_df["response_clean"]].limit(20).toPandas()

In [296]:
# dcard 髒話
dirtywords_path = "/nfs/aq_test/jieba/" + "dcard_dirtywords.txt"
dirtyword_set = set()
with open(dirtywords_path, 'r') as dirtywords:
    for dirtyword in dirtywords:
        dirtyword_set.add(dirtyword.strip('\n').decode('utf-8'))
        
dirty_word_rule = ''
for dw in dirtyword_set:
    dirty_word_rule = dirty_word_rule + dw + '|'

dirty_word_rule = '(' + dirty_word_rule[:len(dirty_word_rule) - 1] + ')'

def has_dirty_word(response):
        
    rule = dirty_word_rule
    pattern = re.compile(rule)
    match = pattern.search(response)
    if match:  
        return True 

    return False

has_dirty_word_udf = udf(has_dirty_word)
data_df = data_df.withColumn("has_dirty_word", 
                             has_dirty_word_udf(lower(data_df['response_clean'])))
data_df.show(5)

In [297]:
# 去除has_url的row or 去除has_floor的row
data_df = data_df.filter((data_df['has_url'] == False) &
                         (data_df['has_reply_tag'] == False) &
                         (data_df['has_dirty_word'] == False)
                         )
print(data_df.count())
data_df.limit(5).show()

In [298]:
data_df.count()

123119

In [299]:
data_df = data_df.select('article_id_2', 
                         'floor',
                         'response', 
                         'response_clean',
                         'response_create_date',
                         'response_crawl_date',
                         'like_count')

In [300]:
response_df = data_df

# 組成query table

In [301]:
# food_article_df food_response_df
join_df = article_df.join(response_df, 
                          article_df.article_id == response_df.article_id_2, 
                          "inner").drop('article_id_2')

# 存到不同版

In [302]:
food_join_df = join_df.filter(join_df.board == "food")
relationship_join_df = join_df.filter(join_df.board == "relationship")

In [303]:
food_join_df.limit(5).toPandas()

,article_id,board,article_crawl_date,article_create_date,title,title_clean,question_type,floor,response,response_clean,response_create_date,response_crawl_date,like_count
0,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,,6,只吃過平丼郎\n真的是宵夜好良伴\n份量也實在,只吃過平丼郎\n真的是宵夜好良伴\n份量也實在,2019-08-03T10:45:58.712Z,2020-04-06 16:50:47.688,1
1,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,,1,感謝分享❤️兩間都想吃😆😆😆,感謝分享❤️兩間都想吃😆😆😆,2019-07-27T07:38:31.197Z,2020-04-06 16:50:47.728,3
2,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,,5,平丼郎好吃～,平丼郎好吃～,2019-07-28T17:33:45.340Z,2020-04-06 16:50:47.738,1
3,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,,2,謝謝分享,謝謝分享,2019-07-27T17:32:16.146Z,2020-04-06 16:50:47.682,1
4,231747478,food,2020-04-06 16:50:45.114,2019-07-27T07:11:14.151Z,屏東丼飯美食嚐鮮篇,屏東丼飯美食嚐鮮篇,,10,市井食堂 也不錯吃,市井食堂 也不錯吃,2019-09-01T06:35:16.430Z,2020-04-06 16:50:45.126,1


In [304]:
# data_df = data_df.filter(data_df.board == "food")
food_join_df.count()

91508

In [307]:
food_join_df.printSchema()

In [312]:
t_dao.saveToCassandraDF(food_join_df, DB_KEYSPACE, dcard_article_response_food, 'overwrite')